# 第七章：使用示例（少样本提示）

- [课程内容](#lesson)
- [练习](#exercises)
- [示例练习场](#example-playground)

## 设置

运行以下设置单元格来加载您的 API 密钥并建立 `get_completion` 辅助函数。

In [ ]:
# 导入 Python 内置的正则表达式库
import re
from openai import OpenAI

# 从 IPython 存储中检索 API_KEY 和 MODEL_NAME 变量
%store -r API_KEY
%store -r MODEL_NAME
%store -r BASE_URL

client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

def get_completion(prompt: str, system_prompt="", prefill=""):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    if prefill:
        messages.append({"role": "assistant", "content": prefill})
    
    response = client.chat.completions.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=messages
    )
    return response.choices[0].message.content

---

## 课程内容

**给模型提供您希望它如何行为（或不希望它如何行为）的示例是极其有效的**，可以用于：
- 获得正确的答案
- 获得正确格式的答案

这种提示方式也被称为"**少样本提示**"（few-shot prompting）。您也可能遇到"零样本"（zero-shot）、"n样本"（n-shot）或"单样本"（one-shot）等术语。"样本"的数量指的是提示中使用了多少个示例。

### 示例

假设您是一个开发者，试图构建一个回答儿童问题的"家长机器人"。**模型的默认响应相当正式和机械化**。这会让孩子感到失望。

In [ ]:
# 提示
PROMPT = "圣诞老人会在圣诞节给我带礼物吗？"

# 打印模型的响应
print(get_completion(PROMPT))

您可以花时间描述您想要的语调，但**给模型提供一些理想响应的示例**会更容易。

In [ ]:
# 提示
PROMPT = """请完成对话，以"A"的身份写出下一行。
Q: 牙仙真的存在吗？
A: 当然存在，亲爱的。把你的牙齿包好放在枕头下面。明天早上可能会有惊喜等着你。
Q: 圣诞老人会在圣诞节给我带礼物吗？"""

# 打印模型的响应
print(get_completion(PROMPT))

在下面的格式化示例中，我们可以逐步指导模型如何提取姓名和职业并按我们想要的方式格式化，或者我们可以**提供一些格式正确的示例，让模型从中推断**。注意`助手`回合中的`<individuals>`，这有助于模型开个好头。

In [ ]:
# 包含可变内容占位符的提示模板
PROMPT = """
#示例1
## 输入
银雾谷，一个迷人的村庄，是一群非凡人物的家园。
其中有李明医生，一位神经外科医生，他在地区医疗中心革新了手术技术。
陈雅是一位创新的建筑师，她用可持续且令人惊叹的设计改变了村庄的景观。
当地剧院因伊万·科瓦奇（一位专业训练的音乐家和作曲家）的迷人交响乐而增色不少。
伊莎贝拉·托雷斯是一位自学成才的厨师，对当地食材充满热情，她创造了一家农场到餐桌的餐厅，成为美食爱好者的必访之地。
这些杰出的个人，每个人都有独特的才能，为银雾谷生活的活力画卷做出了贡献。
## 输出
<individuals>
1. 李明医生 [神经外科医生]
2. 陈雅 [建筑师]
3. 伊万·科瓦奇 [音乐家和作曲家]
4. 伊莎贝拉·托雷斯 [厨师]
</individuals>
#示例2
## 输入
在镇中心，厨师奥利弗·汉密尔顿用他的农场到餐桌餐厅"绿盘子"改变了当地的烹饪场景。奥利弗致力于采购当地有机食材，为这家餐厅赢得了美食评论家和当地人的好评。
沿街而下，您会找到河滨格罗夫图书馆，首席图书管理员伊丽莎白·陈一直努力为所有人创造一个温馨包容的空间。她努力扩展图书馆的服务并为儿童建立阅读项目，对该镇的识字率产生了重大影响。
当您漫步在迷人的镇广场时，您会被装饰墙壁的美丽壁画所吸引。这些杰作是著名艺术家伊莎贝拉·托雷斯的作品，她捕捉河滨格罗夫精髓的才能让这个小镇焕发生机。
河滨格罗夫的体育成就也值得注意，这要归功于前奥运游泳运动员转教练的马库斯·詹金斯。马库斯用他的经验和热情训练镇上的年轻人，带领河滨格罗夫游泳队获得了几个地区冠军。
## 输出
<individuals>
1. 奥利弗·汉密尔顿 [厨师]
2. 伊丽莎白·陈 [图书管理员]
3. 伊莎贝拉·托雷斯 [艺术家]
4. 马库斯·詹金斯 [教练]
</individuals>

橡树谷，一个迷人的小镇，是三位杰出人物的家园，他们的技能和奉献精神对社区产生了持久的影响。
在镇上繁忙的农贸市场，您会找到劳拉·西蒙斯，一位热情的有机农民，以其美味和可持续种植的农产品而闻名。她致力于推广健康饮食，激励镇上拥抱更加环保的生活方式。
在橡树谷的社区中心，凯文·阿尔瓦雷斯是一位技艺精湛的舞蹈教练，为各个年龄段的人们带来了运动的乐趣。他的包容性舞蹈课程在居民中培养了团结感和自我表达，丰富了当地的艺术场景。
最后，雷切尔·奥康纳是一位不知疲倦的志愿者，她将时间奉献给各种慈善活动。她致力于改善他人生活，在橡树谷内创造强烈的社区意识方面起到了重要作用。
通过他们独特的才能和坚定的奉献精神，劳拉、凯文和雷切尔将自己融入了橡树谷的结构中，帮助创造了一个充满活力和繁荣的小镇。"""

# 模型响应的预填充
PREFILL = "<individuals>"

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print("用户输入:")
print(PROMPT)
print("\n助手回应:")
print(PREFILL)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

如果您想在不更改上述任何内容的情况下试验课程提示，请滚动到课程笔记本的最底部访问[**示例练习场**](#example-playground)。

---

## 练习
- [练习 7.1 - 通过示例进行邮件格式化](#exercise-71---email-formatting-via-examples)

### 练习 7.1 - 通过示例进行邮件格式化
我们将重做练习 6.2，但这次，我们将编辑 `PROMPT` 以使用邮件的"少样本"示例加上正确的分类（和格式化）来让模型输出正确答案。我们希望模型输出的*最后一个*字母是类别的字母。

如果您忘记了每封邮件对应的正确字母类别，请参考 `EMAILS` 列表中每封邮件旁边的注释。

请记住，这些是邮件的类别：
- (A) 售前问题
- (B) 损坏或有缺陷的物品
- (C) 账单问题
- (D) 其他（请解释）								

In [ ]:
# 包含可变内容占位符的提示模板
PROMPT = """请将此邮件分类为绿色或蓝色：{email}"""

# 模型响应的预填充
PREFILL = ""

# 作为列表存储的变量内容
EMAILS = [
    "你好 -- 我的Mixmaster4000在操作时发出奇怪的噪音。它还有点烟雾味和塑料味，像燃烧的电子产品。我需要更换。", # (B) 损坏或有缺陷的物品
    "我可以用我的Mixmaster 4000来搅拌油漆吗，还是只能用来搅拌食物？", # (A) 售前问题 OR (D) 其他（请解释）
    "我已经等了4个月，取消后我的月费还没有停止！！到底怎么回事？？？", # (C) 账单问题
    "我怎么到这里的，我不擅长电脑。求助。" # (D) 其他（请解释）
]

# 正确分类存储为列表的列表，以适应每封邮件可能有多个正确分类的情况
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# 遍历邮件列表
for i,email in enumerate(EMAILS):
    
    # 将邮件文本替换到邮件占位符变量中
    formatted_prompt = PROMPT.format(email=email)
   
    # 获取模型的响应
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # 对模型的响应进行评分
    grade = any([bool(re.search(ans, response[-1])) for ans in ANSWERS[i]])
    
    # 打印模型的响应
    print("--------------------------- 包含变量替换的完整提示 ---------------------------")
    print("用户输入")
    print(formatted_prompt)
    print("\n助手回应")
    print(PREFILL)
    print("\n------------------------------------- 模型的响应 -------------------------------------")
    print(response)
    print("\n------------------------------------------ 评分 ------------------------------------------")
    print("此练习是否正确解决:", grade, "\n\n\n\n\n\n")

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
from hints import exercise_7_1_hint; print(exercise_7_1_hint)

仍然遇到困难？运行下面的单元格查看示例解决方案。

In [ ]:
from hints import exercise_7_1_solution; print(exercise_7_1_solution)

### 恭喜！

如果您已经解决了到此为止的所有练习，您就可以进入下一章了。祝您提示愉快！

---

## 示例练习场

这是一个供您自由试验本课程中显示的提示示例的区域，您可以调整提示以查看它如何影响模型的响应。

In [ ]:
# 提示
PROMPT = "圣诞老人会在圣诞节给我带礼物吗？"

# 打印模型的响应
print(get_completion(PROMPT))

In [ ]:
# 提示
PROMPT = """请完成对话，以"A"的身份写出下一行。
Q: 牙仙真的存在吗？
A: 当然存在，亲爱的。把你的牙齿包好放在枕头下面。明天早上可能会有惊喜等着你。
Q: 圣诞老人会在圣诞节给我带礼物吗？"""

# 打印模型的响应
print(get_completion(PROMPT))

In [ ]:
# 包含可变内容占位符的提示模板
PROMPT = """
#示例1
## 输入
银雾谷，一个迷人的村庄，是一群非凡人物的家园。
其中有李明医生，一位神经外科医生，他在地区医疗中心革新了手术技术。
陈雅是一位创新的建筑师，她用可持续且令人惊叹的设计改变了村庄的景观。
当地剧院因伊万·科瓦奇（一位专业训练的音乐家和作曲家）的迷人交响乐而增色不少。
伊莎贝拉·托雷斯是一位自学成才的厨师，对当地食材充满热情，她创造了一家农场到餐桌的餐厅，成为美食爱好者的必访之地。
这些杰出的个人，每个人都有独特的才能，为银雾谷生活的活力画卷做出了贡献。
## 输出
<individuals>
1. 李明医生 [神经外科医生]
2. 陈雅 [建筑师]
3. 伊万·科瓦奇 [音乐家和作曲家]
4. 伊莎贝拉·托雷斯 [厨师]
</individuals>
#示例2
## 输入
在镇中心，厨师奥利弗·汉密尔顿用他的农场到餐桌餐厅"绿盘子"改变了当地的烹饪场景。奥利弗致力于采购当地有机食材，为这家餐厅赢得了美食评论家和当地人的好评。
沿街而下，您会找到河滨格罗夫图书馆，首席图书管理员伊丽莎白·陈一直努力为所有人创造一个温馨包容的空间。她努力扩展图书馆的服务并为儿童建立阅读项目，对该镇的识字率产生了重大影响。
当您漫步在迷人的镇广场时，您会被装饰墙壁的美丽壁画所吸引。这些杰作是著名艺术家伊莎贝拉·托雷斯的作品，她捕捉河滨格罗夫精髓的才能让这个小镇焕发生机。
河滨格罗夫的体育成就也值得注意，这要归功于前奥运游泳运动员转教练的马库斯·詹金斯。马库斯用他的经验和热情训练镇上的年轻人，带领河滨格罗夫游泳队获得了几个地区冠军。
## 输出
<individuals>
1. 奥利弗·汉密尔顿 [厨师]
2. 伊丽莎白·陈 [图书管理员]
3. 伊莎贝拉·托雷斯 [艺术家]
4. 马库斯·詹金斯 [教练]
</individuals>

橡树谷，一个迷人的小镇，是三位杰出人物的家园，他们的技能和奉献精神对社区产生了持久的影响。
在镇上繁忙的农贸市场，您会找到劳拉·西蒙斯，一位热情的有机农民，以其美味和可持续种植的农产品而闻名。她致力于推广健康饮食，激励镇上拥抱更加环保的生活方式。
在橡树谷的社区中心，凯文·阿尔瓦雷斯是一位技艺精湛的舞蹈教练，为各个年龄段的人们带来了运动的乐趣。他的包容性舞蹈课程在居民中培养了团结感和自我表达，丰富了当地的艺术场景。
最后，雷切尔·奥康纳是一位不知疲倦的志愿者，她将时间奉献给各种慈善活动。她致力于改善他人生活，在橡树谷内创造强烈的社区意识方面起到了重要作用。
通过他们独特的才能和坚定的奉献精神，劳拉、凯文和雷切尔将自己融入了橡树谷的结构中，帮助创造了一个充满活力和繁荣的小镇。"""

# 模型响应的预填充
PREFILL = "<individuals>"

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print("用户输入:")
print(PROMPT)
print("\n助手回应:")
print(PREFILL)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))